In [ ]:
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats

In [ ]:
savedir = "/icarus/app/users/gputnam/calib/plots/"
dosave=True

In [ ]:
branches = [
    "meta.run",
    "meta.evt",
    "meta.iproc",
    "t0",
    "id",
    "clear_cosmic_muon",
    "start_x",
    "start_y",
    "start_z",
    "end_x",
    "end_y",
    "end_z",
    "dir_x",
    "dir_y",
    "dir_z",
    "hit_min_time_p2_tpcE",
    "hit_max_time_p2_tpcE",
    "hit_min_time_p2_tpcW",
    "hit_max_time_p2_tpcW"
]

In [ ]:
# fname = "icarus.root"
fname = "/icarus/app/users/gputnam/calib/proc/caloanaout_run5458_N1000_procF_all.root"
folderE = "caloanaCryoE"
folderW = "caloanaCryoW"
tname = "GCalorimetryAnalyzer"

treeE = uproot.open(fname)[folderE][tname]
data = treeE.arrays(branches, library="pd")

In [ ]:
# EXTERNAL INPUT: The drift window in TPC E and TPC W
tcathode_E = 3203.15
tcathode_W = 3212.96

In [ ]:
# CONFIG

a2c_dist = 148.275 # cm
tanode = 850 # ticks
tick_period = 400 # ns
NTICKS = 4096 # ticks

In [ ]:
ICARUS_AV = [    
    {
      "xmin": -364.49,
      "xmax": -67.94,
      "ymin": -173.41,
      "zmin": -909.951,
      "ymax": 143.41,
      "zmax": 879.951,
    },
    {
      "xmin": 67.94,
      "ymin": -173.41,
      "zmin": -909.951,
      "xmax": 364.49,
      "ymax": 143.41,
      "zmax": 879.951,
    }
]

ICARUS_FV = [{
        "xmin": ICARUS_AV[0]["xmin"] + 25,
        "ymin": ICARUS_AV[0]["ymin"] + 25,
        "zmin": ICARUS_AV[0]["zmin"] + 25,
        "xmax": ICARUS_AV[0]["xmax"] - 25,
        "ymax": ICARUS_AV[0]["ymax"] - 50,
        "zmax": ICARUS_AV[0]["zmax"] - 25,
    },
    {
        "xmin": ICARUS_AV[1]["xmin"] + 25,
        "ymin": ICARUS_AV[1]["ymin"] + 25,
        "zmin": ICARUS_AV[1]["zmin"] + 25,
        "xmax": ICARUS_AV[1]["xmax"] - 25,
        "ymax": ICARUS_AV[1]["ymax"] - 50,
        "zmax": ICARUS_AV[1]["zmax"] - 25,
    }
]

def InFV(xyz, FVs=ICARUS_FV):
    fids = []
    fid = xyz[0] == np.nan
    for fv in FVs:
        this_fid = [(xyz[i] > fv[p+"min"]) & (xyz[i] < fv[p+"max"]) for i, p in enumerate("xyz")]
        fid = fid | (this_fid[0] & this_fid[1] & this_fid[2])
    return fid

def InFVYZ(yz, FVs=ICARUS_FV):
    fids = []
    fid = yz[0] == np.nan
    for fv in FVs:
        this_fid = [(yz[i] > fv[p+"min"]) & (yz[i] < fv[p+"max"]) for i, p in enumerate("yz")]
        fid = fid | (this_fid[0] & this_fid[1])
    return fid

def isTPCE(df):
    return df.tpc <= 1


In [ ]:
# use the external input to build the t0
ccross_t0_E = data.hit_max_time_p2_tpcE - tcathode_E
ccross_t0_W = data.hit_max_time_p2_tpcW - tcathode_W

data["ccross_t0"] = ((ccross_t0_E + ccross_t0_W) / 2.) * tick_period

has_t0 = True

In [ ]:
nonfid_track = ~InFV([data.start_x, data.start_y, data.start_z]) &\
    ~InFV([data.end_x, data.end_y, data.end_z])
nonfid_track = nonfid_track & has_t0

nonfid_track.name="nonfid"
nonfid_track.index.name="entry"

#calodatayvalid = calodatay.join(valid_track)

In [ ]:
plane2branches = [
    "x", "y", "z", "time"
]
plane2branches = ["hits2.%s" % s for s in plane2branches]

In [ ]:
plane2_dqdx = treeE.arrays(["hits2.dqdx"], library="pd")#, entrystop=10000)
plane2_oncalo = (plane2_dqdx["hits2.dqdx"] > 0)
plane2_dqdx = plane2_dqdx[plane2_oncalo]
plane2_dqdx["chunk"] = plane2_dqdx.index.get_level_values(1) // 5

calo_plane2 = plane2_dqdx.groupby(["entry", "chunk"])["hits2.dqdx"].median().to_frame()
del plane2_dqdx

for b in plane2branches:
    bdf = treeE.arrays([b], library="pd")
    bdf = bdf[plane2_oncalo]
    bdf["chunk"] = bdf.index.get_level_values(1) // 5
    calo_plane2 = calo_plane2.join(bdf.groupby(["entry", "chunk"])[b].mean())
    del bdf

In [ ]:
# Correct t0
calo_plane2.columns = [s.split(".")[-1] for s in calo_plane2.columns]
calo_plane2.time = (calo_plane2.time * tick_period - data.ccross_t0 - tanode*tick_period) / 1000.

In [ ]:
# Add TPC
plane2_tpc = treeE.arrays(["hits2.tpc"], library="pd")
plane2_tpc = plane2_tpc[plane2_oncalo]
plane2_tpc.columns = [s.split(".")[-1] for s in plane2_tpc.columns]
plane2_tpc["tpcE"] = isTPCE(plane2_tpc)
plane2_tpc["chunk"] = plane2_tpc.index.get_level_values(1) // 5

calo_plane2["tpcE"] = plane2_tpc.groupby(["entry", "chunk"]).tpcE.all()
calo_plane2["tpcW"] = (~calo_plane2.tpcE) & (plane2_tpc.groupby(["entry", "chunk"]).tpcE.nunique() == 1)

del plane2_tpc

In [ ]:
if "nonfid" in calo_plane2.columns:
    calo_plane2 = plane0calo_plane2drop(columns=["nonfid"])
calo_plane2 = calo_plane2.join(nonfid_track)

In [ ]:
calo_plane2 = calo_plane2[calo_plane2.tpcE | calo_plane2.tpcW]
calo_plane2 = calo_plane2.drop(columns=["tpcW"])

In [ ]:
#binx = np.linspace(ICARUS_AV[0]["xmin"], ICARUS_AV[0]["xmin"] + a2c_dist, 26)
tmax = a2c_dist / 0.1574
binx = np.linspace(0, tmax, 21)
biny = np.linspace(200, 800, 26)
when = calo_plane2.tpcE & calo_plane2.nonfid
N, xbin, ybin,_ = plt.hist2d(calo_plane2.time[when], calo_plane2.dqdx[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r", label="Mean")
cbar = plt.colorbar()
cbar.set_label("Entries")

plt.xlabel("Deposition Time [us]")
plt.ylabel("Median dQ/dx [ADC/cm]")
plt.legend(loc="lower left")
plt.title("TPC EE Plane 2")

if dosave: plt.savefig(savedir + "median_dQdx_tpos_EE_p2.png")

In [ ]:
t0 = 0
def exp(t, *p):
    A,tau = p
    return A*np.exp(-(t - t0)/tau)
p0 = [600, 1e3]

where_fit = ~np.isnan(xcenter) & (xcenter > 200)
popt, pcov = curve_fit(exp, xcenter[where_fit], mean[where_fit], p0=p0, sigma=stddev[where_fit])
plt.errorbar(xcenter, mean, yerr=stderr, label="Data")

plt.plot(xcenter, exp(xcenter, *popt), label="Exp Fit")
plt.legend(title="TPC EE")
plt.text(0.05, 0.05, "Electron Lifetime: %.2f [ms]" % (popt[1]/1e3), transform=plt.gca().transAxes, fontsize=14)

plt.xlabel("Deposition Time [us]")
plt.ylabel("Median dQ/dx [ADC/cm]")

if dosave: plt.savefig(savedir + "median_dQdx_xpos_lifetime_EE_p2.png")

In [ ]:
electron_lifetime_tpcE = popt[1]
calo_plane2["dqdx_corr"] = calo_plane2.dqdx * exp(calo_plane2.time, 1., -electron_lifetime_tpcE)

In [ ]:
#binx = np.linspace(ICARUS_AV[0]["xmin"], ICARUS_AV[0]["xmin"] + a2c_dist, 26)
tmax = a2c_dist / 0.1574
binx = np.linspace(0, tmax, 21)
biny = np.linspace(200, 800, 26)
when = ~calo_plane2.tpcE & calo_plane2.nonfid
N, xbin, ybin,_ = plt.hist2d(calo_plane2.time[when], calo_plane2.dqdx[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r", label="Mean")
cbar = plt.colorbar()
cbar.set_label("Entries")

plt.xlabel("Deposition Time [us]")
plt.ylabel("Median dQ/dx [ADC/cm]")
plt.legend(loc="lower left")
plt.title("TPC EW Plane 2")

if dosave: plt.savefig(savedir + "median_dQdx_tpos_EW_p2.png")

In [ ]:
t0 = 0
def exp(t, *p):
    A,tau = p
    return A*np.exp(-(t - t0)/tau)
p0 = [600, 1e3]

where_fit = ~np.isnan(xcenter) & (xcenter > 200)
popt, pcov = curve_fit(exp, xcenter[where_fit], mean[where_fit], p0=p0, sigma=stddev[where_fit])
plt.errorbar(xcenter, mean, yerr=stderr, label="Data")

plt.plot(xcenter, exp(xcenter, *popt), label="Exp Fit")
plt.legend(title="TPC EW")
plt.text(0.05, 0.05, "Electron Lifetime: %.2f [ms]" % (popt[1]/1e3), transform=plt.gca().transAxes, fontsize=14)

plt.xlabel("Deposition Time [us]")
plt.ylabel("Median dQ/dx [ADC/cm]")

if dosave: plt.savefig(savedir + "median_dQdx_tpos_lifetime_EW_p2.png")

In [ ]:
electron_lifetime_tpcW = popt[1]
calo_plane2.loc[~calo_plane2.tpcE, "dqdx_corr"] = calo_plane2.dqdx[~calo_plane2.tpcE] * exp(calo_plane2.time[~calo_plane2.tpcE], 1., -electron_lifetime_tpcW)

In [ ]:
biny = np.linspace(400, 1000, 26)

In [ ]:
binx = np.linspace(ICARUS_AV[0]["xmin"], ICARUS_AV[0]["xmin"] + a2c_dist, 26)

when = calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.x[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()

cbar.set_label("Entries")
plt.xlabel("Deposition X-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EE")

if dosave: plt.savefig(savedir + "median_dQdx_xpos_corrected_EE_p2.png")

In [ ]:
binx = np.linspace(ICARUS_AV[0]["xmax"] - a2c_dist, ICARUS_AV[0]["xmax"], 26)

when = ~calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.x[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()

cbar.set_label("Entries")
plt.xlabel("Deposition X-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EW")

if dosave: plt.savefig(savedir + "median_dQdx_xpos_corrected_EW_p2.png")

In [ ]:
binx = np.linspace(ICARUS_AV[0]["ymin"], ICARUS_AV[0]["ymax"], 26)

when = calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.y[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()

cbar.set_label("Entries")
plt.xlabel("Deposition Y-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EE")

if dosave: plt.savefig(savedir + "median_dQdx_ypos_corrected_EE_p2.png")

In [ ]:
binx = np.linspace(ICARUS_AV[0]["ymin"], ICARUS_AV[0]["ymax"], 26)

when = ~calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.y[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()

cbar.set_label("Entries")
plt.xlabel("Deposition Y-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EW")

if dosave: plt.savefig(savedir + "median_dQdx_ypos_corrected_EW_p2.png")

In [ ]:
binx = np.linspace(ICARUS_AV[0]["zmin"], ICARUS_AV[0]["zmax"], 26)

when = calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.z[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
cbar.set_label("Entries")
plt.xlabel("Deposition Z-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EE")

if dosave: plt.savefig(savedir + "median_dQdx_zpos_corrected_EE_p2.png")


In [ ]:
binx = np.linspace(ICARUS_AV[0]["zmin"], ICARUS_AV[0]["zmax"], 26)

when = ~calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.z[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
cbar.set_label("Entries")
plt.xlabel("Deposition Z-Position [cm]")
plt.ylabel("X-Corr. Median dQ/dx [ADC/cm]")
plt.title("TPC EW")

if dosave: plt.savefig(savedir + "median_dQdx_zpos_corrected_EW_p2.png")


In [ ]:
theta_xz = np.arccos(data.dir_z / np.sqrt(data.dir_x**2 + data.dir_z**2)) * np.sign(data.dir_x)*180/np.pi
theta_xz.name = "theta_xz"
theta_yz = np.arccos(data.dir_z / np.sqrt(data.dir_y**2 + data.dir_z**2)) * np.sign(data.dir_y)*180/np.pi
theta_yz.name = "theta_yz"

In [ ]:
if "theta_xz" in calo_plane2.columns:
    calo_plane2 = calo_plane2.drop(columns=["theta_xz"])
calo_plane2 = calo_plane2.join(theta_xz)

if "theta_yz" in calo_plane2.columns:
    calo_plane2 = calo_plane2.drop(columns=["theta_yz"])
calo_plane2 = calo_plane2.join(theta_yz)

In [ ]:
binx = np.linspace(-180, 180, 19)

when = calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.theta_xz[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
plt.title("TPC EE")

cbar.set_label("Entries")
plt.xlabel("$\\theta_{xz}$ [deg]")
plt.ylabel("$\\tau$-Corr Median dQ/dx [ADC/cm]")
if dosave: plt.savefig(savedir + "median_dQdx_thetaxz_corrected_EE_p2.png")

In [ ]:
binx = np.linspace(-180, 180, 19)

when = ~calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.theta_xz[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
plt.title("TPC EW")

cbar.set_label("Entries")
plt.xlabel("$\\theta_{xz}$ [deg]")
plt.ylabel("$\\tau$-Corr Median dQ/dx [ADC/cm]")
if dosave: plt.savefig(savedir + "median_dQdx_thetaxz_corrected_EW_p2.png")

In [ ]:
binx = np.linspace(-180, 0, 19)

when = calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.theta_yz[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
plt.title("TPC EW")

cbar.set_label("Entries")
plt.xlabel("$\\theta_{yz}$ [deg]")
plt.ylabel("$\\tau$-Corr Median dQ/dx [ADC/cm]")
if dosave: plt.savefig(savedir + "median_dQdx_thetayz_corrected_EE_p2.png")

In [ ]:
binx = np.linspace(-180, 0, 19)

when = ~calo_plane2.tpcE & calo_plane2.nonfid

N, xbin, ybin,_ = plt.hist2d(calo_plane2.theta_yz[when], calo_plane2.dqdx_corr[when], bins=[binx, biny])

ycenter = (ybin[1:] + ybin[:-1]) / 2.
xcenter = (xbin[1:] + xbin[:-1]) / 2.

mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

mean_subtract = np.repeat(mean.reshape((mean.size,1)), ycenter.size, axis=1)
stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
stderr = stddev / np.sqrt(np.sum(N, axis=-1))

plt.errorbar(xcenter, mean, yerr=stderr, color="r")
cbar = plt.colorbar()
plt.title("TPC EW")

cbar.set_label("Entries")
plt.xlabel("$\\theta_{yz}$ [deg]")
plt.ylabel("$\\tau$-Corr Median dQ/dx [ADC/cm]")
if dosave: plt.savefig(savedir + "median_dQdx_thetayz_corrected_EW_p2.png")